# YouNiverse Project
By **AdamantiumForgers**

*Study of the politic polarization in various topics (based on big media followers) : Analyze vocabulary in videos posted (Title and description), How the videos are reveived (likes, dislikes, views),list of user_id and their behaviour in the platform, coverage of events (Elections, war)*

# Dataset
YouNiverse comprises metadata from over 136k channels and 72.9M videos (English-Speaking only!) published between May 2005 and October 2019, as well as channel-level time-series data with weekly subscriber and view counts.

Source: https://zenodo.org/record/4650046 \
Github: https://github.com/epfl-dlab/YouNiverse \
Size: 111 GB (compressed, in total)

**List of files**
* df_channels_en.tsv.gz (6MB): List of the 136'471 channels with some infos (state in october 2019)
* df_timeseries_en.tsv.gz (571MB): Weekly timeseries for each channel, from 03 July 2017 to 23 October 2019
* num_comments.tsv.gz (755MB): List of videos (display_id) with their number of comments
* num_comments_authors.tsv.gz (1.4GB):  ?????????????????????
* youtube_comments.tsv.gz (77.2GB): ~8.6B comments made by ~449M users in 20.5M videos. Each rows = 1 comment: user id, a video id, number of replies and likes the comment received
* yt_metadata_en.jsonl.gz (13.6GB): metadata data related to ~73M videos from ~137k channels
* yt_metadata_helper.feather (2.8GB): Same as jsonl except description, tags, and title (the largest fields)

# Packages

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import os
import json
#import glob
#import gzip
#import swifter
#import langdetect
#import zstandard as zstd
#import matplotlib as mpl
#import scipy.stats as stats
#import matplotlib.pyplot as plt
#import matplotlib.ticker as mtick
#from matplotlib.lines import Line2D
#import matplotlib.font_manager as font_manager



In [2]:
#class zreader:
#
#    def __init__(self, file, chunk_size=16384):
#        self.fh = open(file, 'rb')
#        self.chunk_size = chunk_size
#        self.dctx = zstd.ZstdDecompressor()
#        self.reader = self.dctx.stream_reader(self.fh)
#        self.buffer = ''
#
#    def readlines(self):
#        while True:
#            chunk = self.reader.read(self.chunk_size).decode("utf-8", errors="ignore")
#            if not chunk:
#                break
#            lines = (self.buffer + chunk).split("\n")
#
#            for line in lines[:-1]:
#                yield line
#
#            self.buffer = lines[-1]

## Paths

In [2]:
# Directories for the data.
# Large data can be stored on an external drive and accessed by creating a simlink to a "large" directory in the data folder
DIR = "data/"
DIR_LARGE = "data/large/"

# Path for each file
PATH_TIME_SERIES = DIR + "df_timeseries_en.tsv.gz"
PATH_CHANNELS = DIR + "df_channels_en.tsv.gz"
PATH_NUM_COMMENTS = DIR + "num_comments.tsv.gz"
PATH_NUM_COMMENTS_AUTHORS = DIR + "num_comments_authors.tsv.gz"
PATH_METADATA = DIR_LARGE + "yt_metadata_en.jsonl.gz"
PATH_METADATA_HELPER = DIR + "yt_metadata_helper.feather"
PATH_COMMENTS = DIR_LARGE + "youtube_comments.tsv.gz"

## Load data

In [60]:
df_channels = pd.read_csv(PATH_CHANNELS, compression="infer", sep="\t", nrows=10000)
df_timeseries = pd.read_csv(PATH_TIME_SERIES, compression="infer", sep="\t", nrows=10000)
#df_comments = pd.read_csv(PATH_COMMENTS, compression="infer", sep="\t", nrows=10000)
df_num_comments = pd.read_csv(PATH_NUM_COMMENTS, compression="infer", sep="\t", nrows=10000)
df_num_comments_authors = pd.read_csv(PATH_NUM_COMMENTS_AUTHORS, compression="infer", sep="\t", nrows=10000)

#df_metadata = pd.read_json(PATH_METADATA, lines=True, nrows=100)
#df_metadata_light = pd.read_feather(PATH_METADATA_HELPER,[1,5]) #Only 1:channel_id and 5:like_count

## Preprocess date fields
df_channels["join_date"] = pd.to_datetime(df_channels["join_date"])
#df_timeseries["datetime"] = pd.to_datetime(df_timeseries["datetime"])



In [6]:
#reader = Zreader("PATH_COMMENTS", chunk_size=16384)

In [88]:
df_channels_filtered = df_channels[(df_channels['subscribers_cc'] >= 1e6) & (df_channels['category_cc']=='News & Politics')]
display(df_channels_filtered)

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
129,News & Politics,2006-08-26,UCttspZesZIDEwwpVIgoZtWQ,IndiaTV,15177282,139814,199.0,2.0870
133,News & Politics,2012-06-01,UCRWFSbif-RFENbBrSiez1DA,ABP NEWS,16274836,129027,207.0,2.0870
212,News & Politics,2017-03-03,UCmphdqZNmqL72WJ2uyiNw5w,ABP NEWS HINDI,10800000,51298,340.0,2.0870
268,News & Politics,2015-03-23,UCx8Z14PpntdaxCt2hakbQLQ,The Lallantop,9120000,9423,438.0,2.0870
337,News & Politics,2007-06-19,UCIvaYmXn910QMdemBG3v1pQ,Zee News,9280000,102648,549.0,2.0870
...,...,...,...,...,...,...,...,...
6058,News & Politics,2012-07-27,UCKZGcrxRAhdUi58Mdr565mw,African Diaspora New...,1100000,3576,14705.0,2.8205
6098,News & Politics,2006-09-18,UC52X5wxOL_s5yw0dQk7NtgA,Associated Press,1040000,129072,14780.0,2.7865
6205,News & Politics,2016-11-13,UCYzNg69k1vt9oPtcYJ1kuIg,Ravish Kumar Fan Clu...,1007864,431,15107.0,2.7840
6412,News & Politics,2011-02-13,UCDATi9UrhqjfVNxIKNizOiQ,Jason A,1060000,259,15693.0,2.8280


In [86]:
#del result
result = pd.DataFrame()

for chunk in pd.read_csv(PATH_CHANNELS, compression="infer", sep="\t", chunksize=10000):
    # For example, filter all channels with more than 10M subs
    df_temp = chunk[chunk['subscribers_cc'] > 1e6]
    result = pd.concat([result, df_temp])
        
    df_result = pd.DataFrame(result)

display(df_result)

#for chunk in pd.read_csv(PATH_CHANNELS, compression="infer", sep="\t", chunksize=10000):
#    n_subscribers = chunk["subscribers_cc"]
#    chunk_result = n_subscribers.value_counts()
#    if result is None:
#        result = chunk_result
#    else:
#        result = result.add(chunk_result, fill_value=0)µ

#result.sort_values(ascending=False, inplace=True)

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,Gaming,2010-04-29,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,101000000,3956,3.0,2.0870
1,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0,2.0870
2,Entertainment,2006-09-20,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,56018869,32661,8.0,2.0870
3,Howto & Style,2016-11-15,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,60600000,3591,9.0,2.0870
4,Sports,2007-05-11,UCJ5v_MCY6GNUBTO8-D3XoAg,WWE,48400000,43421,11.0,2.0870
...,...,...,...,...,...,...,...,...
6847,Entertainment,2012-01-02,UCOPbsY_gLA4vuzN8fGOf2Qw,FaZe Nikan,1010000,211,16949.0,2.8445
6852,Howto & Style,2007-12-04,UCcZ2nCUn7vSlMfY5PoH982Q,Still GlamorUs,1010000,680,16966.0,2.8445
6888,Entertainment,2017-03-22,UCJpNhxu82LYcPvnK85yQH3Q,Lisa Luke Nursery Rh...,1009111,19,17117.0,2.8625
6944,Entertainment,2015-03-04,UCLO5YnTRG8jySOUeU2ohnpw,Tristan Tales,1010000,140,17278.0,2.8200


In [ ]:
filtered_metadata = pd.DataFrame()

for chunk in pd.read_json(PATH_METADATA, lines=True, compression="infer", chunksize=10000):
    # For example, filter all channels with more than 10M subs
    df_temp = chunk[chunk['channel_id'].isin(df_channels_filtered['channel'])]
    filtered_metadata = pd.concat([filtered_metadata, df_temp])


filtered_metadata.head(5)
#print(len(df_result))

In [7]:
df_metadata_light.head()

NameError: name 'df_metadata_light' is not defined

In [23]:
df_comments.head()

,author,video_id,likes,replies
0,1,Gkb1QMHrGvA,2,0
1,1,CNtp0xqoods,0,0
2,1,249EEzQmVmQ,1,0
3,1,_U443T2K_Bs,0,0
4,1,rJbjhm0weYc,0,0


In [46]:
df_num_comments[df_num_comments['num_comms']>=10000].head()

,display_id,num_comms
4314,luHmXHEpF7w,19337.0
5137,i2dTXB3O3dw,11340.0
5766,x7niulDayUs,16891.0
6122,-zf7UxfL0zE,11286.0
6892,RZqaeusdMTk,26497.0


In [26]:
df_num_comments_authors.head()

,author,video_id
0,1,5
1,2,3
2,3,2
3,4,6
4,5,3


In [55]:
df_timeseries

,channel,category,datetime,views,delta_views,subs,delta_subs,videos,delta_videos,activity
0,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-03 00:00:00,2.024946e+05,0.000000,650.222222,0.000000,5,0,3
1,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-10 00:00:00,3.940857e+05,191591.111111,1046.000000,395.777778,6,1,1
2,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-17 00:00:00,8.353938e+05,441308.083333,1501.500000,455.500000,6,0,1
3,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-24 00:00:00,1.104577e+06,269183.250000,1750.000000,248.500000,6,0,0
4,UCBJuEqXfXTdcPSbGO9qqn1g,Film and Animation,2017-07-31 00:00:00,1.284406e+06,179828.600000,2008.300000,258.300000,6,0,0
...,...,...,...,...,...,...,...,...,...,...
9995,UCPascXL4VviE9wHLsepiEZg,Music,2019-09-23 00:00:00,2.287771e+07,325092.619048,24885.714286,278.047619,187,0,0
9996,UCZ3rO8Nb9I1p9Wh_lQ6ebew,Howto & Style,2017-04-17 00:00:00,1.500925e+04,1786.250000,310.250000,0.000000,31,0,8
9997,UCZ3rO8Nb9I1p9Wh_lQ6ebew,Howto & Style,2017-04-24 00:00:00,1.715900e+04,2149.750000,349.500000,39.250000,37,6,9
9998,UCZ3rO8Nb9I1p9Wh_lQ6ebew,Howto & Style,2017-05-01 00:00:00,1.931438e+04,2155.375000,385.375000,35.875000,39,2,9


In [32]:
df_metadata

,categories,channel_id,crawl_date,description,dislike_count,display_id,duration,like_count,tags,title,upload_date,view_count
0,Film & Animation,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:26.270363,Lego City Police Lego Firetruck Cartoons about...,1,SBqSc91Hn9g,1159,8,"lego city,lego police,lego city police,lego ci...",Lego City Police Lego Firetruck Cartoons about...,2016-09-28 00:00:00,1057
1,Film & Animation,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:26.914516,Lego Marvel SuperHeroes Lego Hulk Smash Iron-M...,1,UuugEl86ESY,2681,23,"Lego superheroes,lego hulk,hulk smash,lego mar...",Lego Marvel SuperHeroes Lego Hulk Smash Iron-M...,2016-09-28 00:00:00,12894
2,Film & Animation,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:26.531203,Lego City Police Lego Fireman Cartoons about L...,779,oB4c-yvnbjs,1394,1607,"lego city,lego police,lego city police,lego fi...",Lego City Police Lego Fireman Cartoons about L...,2016-09-28 00:00:00,1800602
3,Film & Animation,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:28.335329,Lego Harry Potter Complete Lego New Movie for ...,24,ZaV-gTCMV8E,5064,227,"Lego harry potter,new harry potter,harry potte...",Lego Harry Potter Complete Lego New Movie for ...,2016-09-28 00:00:00,57640
4,Film & Animation,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:30.328487,Lego City Police LONG VIDEO for kids Lego Fire...,13,cGvL7AvMfM0,3554,105,"lego city,lego police,lego city police,lego fi...",Lego City Police 1 HOUR LONG VIDEO for kids Le...,2016-09-28 00:00:00,86368
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Film & Animation,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:29.927636,Lego City Police Lego Fireman Firetruck for Ki...,0,Fa-pX0r76Ww,1604,12,"lego city,lego city police,lego fireman,lego b...",Lego City Police Lego Fireman Firetruck for Ki...,2016-09-05 00:00:00,5869
96,Film & Animation,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:31.849803,Lego City Police Lego Firetruck Fireman Cartoo...,2,E_cZHP4xN5w,3565,19,"Lego city police,lego city,bomberos para niños...",Lego City Police 1 HOUR Lego Firetruck Fireman...,2016-09-04 00:00:00,10215
97,Film & Animation,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:33.781277,Lego City Fireman Lego Police Firetruck Cartoo...,1,aoKUiMQQXtE,2997,13,"lego city police,lego police,lego city,lego po...",Lego City Police Fireman Lego Firetruck Cartoo...,2016-09-04 00:00:00,7676
98,Gaming,UCzWrhkg9eK5I8Bm3HfV-unA,2019-10-31 20:19:27.966732,Lego City Fireman Lego Police Firetruck Cartoo...,0,ZS2TwbEzQTk,1822,8,"lego city police,lego fireman,lego firetruck,l...",Lego City Fireman Lego Police Firetruck Cartoo...,2016-09-04 00:00:00,3560


In [8]:
# Sort by date
df_channels.sort_values(ascending=[True],by=['join_date']).head(20)

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
192,Entertainment,2005-06-16,UCvC4D8onUfXzvjTOM-dBfEA,Marvel Entertainment...,13200000,6332,302.0,2.0870
2396,Film and Animation,2005-06-20,UCiCnPY--pbn5S8JkJdV2PbQ,Bengali Movies - Ang...,2312489,9659,5101.0,2.3905
299,Science & Technology,2005-06-22,UCE_M8A5yxnLfW0KghEeajjw,Apple,9970000,291,489.0,2.0870
1764,Comedy,2005-07-19,UC7RGwnsSAEClqqxJ9MxUhyQ,Lucas,3100000,403,3648.0,2.2735
9219,Gaming,2005-08-04,UCbrLdiQyWBiFxWPLW3oI9Sw,Mikey Gaming,676000,1216,23820.0,2.9480
1663,Entertainment,2005-08-27,UCZGYJFUizSax-yElQaFDp5Q,Star Wars,2950000,1218,3394.0,2.2930
1895,Music,2005-09-06,UCoRUmyNL8KEYftmxRA2n3SQ,Kelly Clarkson,2870000,179,3945.0,2.3040
4171,Music,2005-09-08,UCGPI7zCpQYpPhDLdOAmpAww,rebecca,1460000,241,9634.0,2.5910
5439,Music,2005-09-13,UCaisXKBdNOYqGr2qOXCLchQ,Iron Maiden,1134309,209,12986.0,2.7435
1628,Gaming,2005-09-16,UCFctpiB_Hnlk3ejWfHqSm6Q,The Official Pokémo...,3110000,1161,3318.0,2.2925


In [9]:
df_channels.query('subscribers_cc > 1e7')

,category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
0,Gaming,2010-04-29,UC-lHJZR3Gqxm24_Vd_AJ5Yw,PewDiePie,101000000,3956,3.0,2.087
1,Education,2006-09-01,UCbCmjCuTUZos6Inko4u57UQ,Cocomelon - Nursery ...,60100000,458,7.0,2.087
2,Entertainment,2006-09-20,UCpEhnqL0y41EpW2TvWAHD7Q,SET India,56018869,32661,8.0,2.087
3,Howto & Style,2016-11-15,UC295-Dw_tDNtZXFeAPAW6Aw,5-Minute Crafts,60600000,3591,9.0,2.087
4,Sports,2007-05-11,UCJ5v_MCY6GNUBTO8-D3XoAg,WWE,48400000,43421,11.0,2.087
...,...,...,...,...,...,...,...,...
324,Music,2012-01-23,UCpx_k19S2vUutWUUM9qmXEg,ImagineDragonsVEVO,10300000,108,533.0,2.087
331,Entertainment,2013-04-24,UCwjoPtSoNLAoX2sLBaKLYng,Toys And Funny Kids ...,10200000,3655,543.0,2.087
333,Entertainment,2009-06-08,UCc6W7efUSkd9YYoxOnctlFg,Bethany Mota,10200000,483,545.0,2.087
339,Comedy,2006-12-01,UCazMm3tOCkYrIGE_17j0mVg,Bart Baker,10100000,288,553.0,2.087


To read feather files: https://arrow.apache.org/docs/python/feather.html#:~:text=Feather%20is%20a%20portable%20file,Python%20(pandas)%20and%20R.


In [29]:
df_channels.iloc[10]
#df_channels_politics = df_channels[df_channels['category_cc' == 'Music']]
#df_channels.query('category_cc == Music')

category_cc                             Gaming
join_date                           2015-04-06
channel               UCEdvpU2pFRCVqU6yIPyTpMQ
name_cc                             Marshmello
subscribers_cc                        39100000
videos_cc                                  366
subscriber_rank_sb                        18.0
weights                                  2.087
Name: 10, dtype: object